In [1]:
import dgl
import pandas as pd
import numpy as np
import torch 
import os
import sys
sys.path.append('./../..')
sys.path.append('./..')
from tqdm import tqdm
import argparse
import pandas as pd
import numpy as np
import sys
sys.path.append('./..')
sys.path.append('./../..')
from pathlib import Path
import os
from common_utils import utils
from joblib import Parallel,delayed
import multiprocessing as mp
import pickle 
from torch import LongTensor as LT
from torch import FloatTensor as FT
from torch.nn import functional as F
import dgl.function as fn
from torch import nn
DIR = 'us_import1'
# Use training data for graph creation
DATA_LOC = './../generated_data_v1/{}'.format(DIR)




with open(os.path.join(DATA_LOC,'domain_dims.pkl'),'rb') as fh:
    domain_dims = pickle.load(fh) 
    
    

Using backend: pytorch
/home/ddatta/anaconda3/envs/graph1/lib/python3.7/site-packages/dgl/base.py:45: DGLWarning: Detected an old version of PyTorch. Suggest using torch>=1.5.0 for the best experience.
  return warnings.warn(message, category=category, stacklevel=1)


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
# ----------------------------
# First create a bipartite graph using training data  
# 2 entity types : Consignee and Shipper
# Prefixes : 'Consignee' 'Shipper'
# -----------------------------

# ----------------------------
attr_consignee_prefix = 'ConsigneePanjivaID'
attr_shipper_prefix = 'ShipperPanjivaID'
bipartite_domains = sorted([attr_consignee_prefix, attr_shipper_prefix])

In [3]:
def get_data_df():
    global attr_consignee_prefix
    global attr_shipper_prefix
    global DATA_LOC
    global bipartite_domains
    global DIR
    
    with open(os.path.join(DATA_LOC,'domain_dims.pkl'),'rb') as fh:
        domain_dims = pickle.load(fh)
        
    # This data does not have serial ID but domain specific ID
    data_df = pd.read_csv(os.path.join(DATA_LOC, 'train_data.csv'),index_col=None, low_memory=False)
    data_df = data_df.drop_duplicates(subset=list(domain_dims.keys()))
    # group by
    g_df = data_df.groupby([attr_consignee_prefix, attr_shipper_prefix]).size().reset_index(name='weight')
    
    # -----------------------------
    # Create synthetic mappiing 
    # -----------------------------
  
    
    synID = 0
    cur = 0
    col_syn_id = []
    col_entity_id = []
    col_domain_names = []
    # ------------------
    for d in sorted(bipartite_domains):
        s = domain_dims[d]
        col_entity_id.extend(list(range(s)))
        col_domain_names.extend([d for _ in range(s)])
        tmp = np.arange(s) + cur
        tmp = tmp.tolist()
        col_syn_id.extend(tmp)
        cur += s

    data = {'domain': col_domain_names, 'entity_id': col_entity_id, 'syn_id': col_syn_id}
    synID_mapping_df = pd.DataFrame(data)
    
    # -------------------
    # Replace entity_id with synthetic id 
    # -------------------
    mapping_dict = {}
    for domain in set(synID_mapping_df['domain']):
        tmp =  synID_mapping_df.loc[(synID_mapping_df['domain'] == domain)]
        syn_id = tmp['syn_id'].values.tolist()
        entity_id = tmp['entity_id'].values.tolist()
        mapping_dict[domain] = {k:v for k,v in zip(entity_id,syn_id)}
        
    def convert_aux(val, domain):
        return mapping_dict[domain][val]

    for domain in tqdm(bipartite_domains):
        g_df[domain] = g_df[domain].parallel_apply(convert_aux, args=(domain,))
        
        
    return g_df, synID_mapping_df, data_df


In [4]:
g_df, synID_mapping_df, data_df = get_data_df()

100%|██████████| 2/2 [00:01<00:00,  1.05it/s]


In [5]:
g_df.head(10)

,ConsigneePanjivaID,ShipperPanjivaID,weight
0,0,4947,1
1,0,5336,6
2,0,5340,17
3,0,5943,7
4,0,7339,2
5,0,7552,2
6,0,8142,6
7,0,8754,14
8,1,5267,1
9,1,6015,1


In [6]:
# ----------------------------------
# Read In metapath2vec features  
# Those are stored as numpy arrays
# ---------------------------------.
mp2v_features = {}
MP2V_features_LOC = os.path.join('./../createGraph_trade/saved_model_data', DIR)
for attr in bipartite_domains:
    file = os.path.join(MP2V_features_LOC, 'mp2v_{}.npy'.format(attr))
    mp2v_features[attr] = np.load(file)
    
'''
To feed data into DGL graph 
Create tensor of features
'''
input_emb_size = list(mp2v_features.values())[0].shape[1]
num_entities = len(synID_mapping_df)
input_features = np.zeros([num_entities, input_emb_size])




In [31]:
for d in bipartite_domains:
    tmp = synID_mapping_df.loc[synID_mapping_df['domain']==d]
    _syn_id = tmp['syn_id'].values.tolist()
    _entity_id = tmp['entity_id'].values.tolist()
    input_features[_syn_id] = mp2v_features[d][_entity_id]

In [33]:
input_features[:10]

array([[ 0.52414888, -0.45322031,  1.91028905, ...,  1.29858494,
         2.42626166,  2.1162219 ],
       [ 0.06493711, -1.25776231,  2.04438019, ..., -0.46997485,
         1.6251533 , -0.40729418],
       [-0.8736642 ,  0.93814248, -0.60021335, ...,  0.48887551,
        -0.15444292,  0.80477917],
       ...,
       [-0.90004742,  1.25017679,  1.40393507, ..., -0.14855796,
         2.29815888, -0.50512475],
       [-2.80783772,  4.58135986, -0.11078735, ..., -1.54375505,
         1.16891015,  0.63428003],
       [-0.71271372,  1.44254029,  0.34384501, ..., -0.98573357,
         1.17151284, -0.87737447]])

In [54]:
src = g_df[attr_consignee_prefix].values.tolist()
dst =  g_df[attr_shipper_prefix].values.tolist()

nodes_src = src + dst
nodes_dst = dst + src
weights = g_df['weight'].values.tolist() +  g_df['weight'].values.tolist()
# DGL is not undirected by default 
graph_obj = dgl.graph((nodes_src,nodes_dst))
graph_obj.edata['weight'] = FT(weights)
graph_obj.ndata['mp2v'] = FT(input_features) 


# graph_obj = dgl.to_bidirected(graph_obj, copy_ndata=True)


In [55]:
graph_obj.ndata

{'mp2v': tensor([[ 0.5241, -0.4532,  1.9103,  ...,  1.2986,  2.4263,  2.1162],
        [ 0.0649, -1.2578,  2.0444,  ..., -0.4700,  1.6252, -0.4073],
        [-0.8737,  0.9381, -0.6002,  ...,  0.4889, -0.1544,  0.8048],
        ...,
        [-0.6626,  0.8059, -1.8143,  ...,  0.8304, -0.6618,  1.1085],
        [-0.9119,  1.6221, -0.9209,  ..., -0.3967,  0.4786, -0.1086],
        [-0.8543, -0.6464, -2.5652,  ..., -0.3752,  0.0170, -0.8392]])}

In [71]:
# ------------------------------

# ------------------------------
'''
Core GNN 
'''

class GNN_1(torch.nn.Module):
    def __init__(
        self,
        inp_dim,
        out_dim,
        input_feature_label='mp2v', 
        aggregator_type='mean',
        bias=True,
        norm=True,
        activation=None
    ):
        super(GNN_1, self).__init__()
        self.out_dim = out_dim
        self.inp_dim = inp_dim
        self.aggregator_type = aggregator_type
        self.norm = norm
        self.activation = activation
        # ------
        # Assuming a 2 layer GNN
        # ------
        self.num_layers = 2 
        self.input_feature_label= input_feature_label
        self.FC_w = nn.Linear(
            self.inp_dim * 3, 
            self.out_dim, 
            bias=bias
        )
        
        self.reset_parameters()
        self.aggregator_type = aggregator_type
        return
    
    def reset_parameters(self):
        """Reinitialize learnable parameters."""
        gain = nn.init.calculate_gain('relu')
        nn.init.xavier_uniform_(
            self.FC_w.weight, 
            gain=gain
        )
        return
    
    
    def forward(self, graph):   
        
        for layer in range( self.num_layers ):
            
            # --------------------------------------
            # If first layer store the result from h
            # --------------------------------------
            if layer == 0 :
                print('Updating Layer 0')
                # For the 1st layer initilize features with input feature
                graph_obj.ndata['features'] = graph_obj.ndata[self.input_feature_label]
                graph_obj.update_all(
                    fn.copy_u('features', 'm'),
                    fn.mean('m', 'h')
                )
                
            else:
                print('Updating Layer 1 a')
                # ------------------------------
                # Update along the given edge
                # ------------------------------
                graph_obj.update_all(
                    fn.copy_u('features', 'm'),
                    fn.mean('m', 'h')
                )
                print('Updating Layer 1b')
                concat_ftrs = torch.cat([
                    graph_obj.ndata['features'],
                    graph_obj.ndata['h'],
                    graph_obj.ndata[self.input_feature_label]
                    ], 
                    dim=1
                )
                
                ftrs = self.FC_w(concat_ftrs)
                ftrs = F.tanh(ftrs)
                if self.norm:
                    ftrs = ftrs/torch.norm(ftrs, p=2)
                graph_obj.ndata['features'] = ftrs 
        return
    
    
    

In [72]:
gnn_obj = GNN_1(
    inp_dim = 128,
    out_dim = 32
)

In [69]:
# --------------------------------------
# Triplet loss for training the GNN
# --------------------------------------
def triplet_loss( xt, xp, xn):
    m = 0.25
    v1 = 1 - F.cosine_similarity(xt, xp, dim=1)
    v2 = 1 - F.cosine_similarity(xt, xn, dim=1)
    d =  v1 - v2 + m
    loss = torch.max(d, torch.zeros_like(d))
    return torch.mean(loss, dim=0, keepdim=False)

In [12]:
def generate_pos_neg_neighbors(
    graph_obj
):
 
    # The source nodes 
    nodes = graph_obj.nodes().data.numpy()
    edges = graph_obj.edges()
    node_src = edges[0].data.numpy()
    node_dest = edges[1].data.numpy()
    
    pos_nbrs = { _ : [] for _ in nodes }
    neg_nbrs = { _ : [] for _ in nodes }
    
    for i,j  in zip(node_src, node_dest):
        pos_nbrs[i].append(j)
    
    # Stor the list of neighbors as values in the dictionary
    neg_nbr_dict = {}
    pos_nbr_dict = {}
    print(pos_nbrs[100])
 

    for node in nodes:
        pos_nbrs[node] = set(pos_nbrs[node])
        neg_nbrs[node] = list(set(nodes).difference(pos_nbrs[node]))
       
        neg_nbr_dict[node] = list(neg_nbrs[node])
        pos_nbr_dict[node] = list(pos_nbrs[node])
        
    return pos_nbr_dict,neg_nbr_dict

In [13]:
def generate_training_triplets(pos_nbr_dict,neg_nbr_dict):
    nodes = list(pos_nbr_dict.keys())
    pos_pairs = []
    neg_pairs = []
    triplets = []
    for n in nodes:
        t = [ n, 
            np.random.choice(pos_nbr_dict[n],size=1)[0], 
            np.random.choice(neg_nbr_dict[n],size=1)[0]
        ]
        triplets.append(t)
    triplets = np.array(triplets)
    return triplets

In [14]:
graph_obj.edges()[0].data.numpy()

array([    0,     0,     0, ..., 10010, 10010, 10011])

In [15]:
pos_nbr_dict,neg_nbr_dict = generate_pos_neg_neighbors(graph_obj)

[8565, 8667]


In [17]:
triplets = generate_training_triplets(pos_nbr_dict,neg_nbr_dict)



In [58]:
graph_obj.ndata['mp2v']

tensor([[ 0.5241, -0.4532,  1.9103,  ...,  1.2986,  2.4263,  2.1162],
        [ 0.0649, -1.2578,  2.0444,  ..., -0.4700,  1.6252, -0.4073],
        [-0.8737,  0.9381, -0.6002,  ...,  0.4889, -0.1544,  0.8048],
        ...,
        [-0.6626,  0.8059, -1.8143,  ...,  0.8304, -0.6618,  1.1085],
        [-0.9119,  1.6221, -0.9209,  ..., -0.3967,  0.4786, -0.1086],
        [-0.8543, -0.6464, -2.5652,  ..., -0.3752,  0.0170, -0.8392]])

In [67]:
def train_model(
    graph_obj, 
    gnn_obj,
    pos_nbr_dict,
    neg_nbr_dict,
    num_epochs = 10
):
    opt = torch.optim.Adam(
        list(gnn_obj.parameters())
    )
    
    pbar = tqdm(range(num_epochs))
    for e in pbar :
        opt.zero_grad()
        triplets = generate_training_triplets(pos_nbr_dict,neg_nbr_dict)
       
        gnn_obj(graph_obj)
        idx_t = LT(triplets[:,0])
        emb_t = graph_obj.ndata['features'][idx_t,:] 
       
        idx_p = LT(triplets[:,1])
        idx_n = LT(triplets[:,2])
        emb_p = graph_obj.ndata['features'][idx_p,:] 
        emb_n = graph_obj.ndata['features'][idx_n,:] 
        loss_val = triplet_loss(emb_t, emb_p, emb_n)
        loss_val.backward()
        opt.step()
#         print(' Epoch {} Loss {:4f}'.format(e,np.mean(loss_val.data.numpy())))
        pbar.set_postfix({'Loss': '{:4f}'.format(np.mean(loss_val.data.numpy()))})

    return

In [73]:
train_model(
    graph_obj, 
    gnn_obj,
    pos_nbr_dict,
    neg_nbr_dict,
    num_epochs = 10 
)


 10%|█         | 1/10 [00:12<01:50, 12.26s/it, Loss=0.196880]

Updating Layer 0
Updating Layer 1 a
Updating Layer 1b


 20%|██        | 2/10 [00:24<01:38, 12.34s/it, Loss=0.186077]

Updating Layer 0
Updating Layer 1 a
Updating Layer 1b


 30%|███       | 3/10 [00:37<01:26, 12.35s/it, Loss=0.172316]

...

Updating Layer 0
Updating Layer 1 a
Updating Layer 1b


 90%|█████████ | 9/10 [01:52<00:12, 12.70s/it, Loss=0.114758]

Updating Layer 0
Updating Layer 1 a
Updating Layer 1b


100%|██████████| 10/10 [02:04<00:00, 12.49s/it, Loss=0.108822]

Updating Layer 0
Updating Layer 1 a
Updating Layer 1b
